In [ ]:
!pip install mlflow boto3 dagshub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 123.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.0/262.0 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 119.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131

In [ ]:
## step 2: set up the mlflow tracking server
import dagshub
import mlflow
dagshub.init(repo_owner='codex03080', repo_name='reddit-sentiement-analysis', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=e682647f-bef6-4d27-8caf-50163eb72235&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=e37f7bb0d2a5e9db7983077ec3d3f7822426fadcfdf9867f9225df43998a2cbd




Accessing as codex03080

Initialized MLflow to track repo "codex03080/reddit-sentiement-analysis"

Repository codex03080/reddit-sentiement-analysis initialized!

In [ ]:
## Step 2: set or create an experiment
mlflow.set_experiment("Ex 2 - Bow vs tfidf")

<Experiment: artifact_location='mlflow-artifacts:/046c7d4d6f8e4fc8a1867a636772c259', creation_time=1770696248317, experiment_id='2', last_update_time=1770696248317, lifecycle_stage='active', name='Ex 2 - Bow vs tfidf', tags={}>

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [ ]:
df = pd.read_csv("/content/reddit_preprocessing.csv").dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [ ]:
## Step 1: Function run the experiment
def run_experiment(vectorizer_type,ngram_range,vectorizer_max_features,vectorizer_name):
  ## Step 2: Vectorization
  if vectorizer_type == "BOW":
    vectorizer = CountVectorizer(ngram_range=ngram_range,max_features=vectorizer_max_features)
  else:
    vectorizer = TfidfVectorizer(ngram_range=ngram_range,max_features=vectorizer_max_features)

  X_train,X_test,y_train,y_test = train_test_split(df['clean_comment'],df['category'],test_size=0.2,random_state=42)

  X_train = vectorizer.fit_transform(X_train)
  X_test = vectorizer.transform(X_test)

  ## Step 4: Define and train a Random Forest Model
  with mlflow.start_run() as run:
    ## set tags for experiment and run
    mlflow.set_tag("mlflow.runName",f"{vectorizer_name}_{ngram_range}_RandomForest")
    mlflow.set_tag("experiment_type","feature_extraction")
    mlflow.set_tag("model_type","RandomForestClassifier")

    ## Add a description
    mlflow.set_tag("description",f"RandomForest with {vectorizer_name},ngram_range={ngram_range},max_feature={vectorizer_max_features}")

    ## Log vectorizer parameters
    mlflow.log_param("vectorizer_type",vectorizer_type)
    mlflow.log_param("ngram_range",ngram_range)
    mlflow.log_param("vectorizer_max_features",vectorizer_max_features)

    ## Log Random Forest parameters
    n_estimators = 200
    max_depth = 15

    mlflow.log_param("n_estimators",n_estimators)
    mlflow.log_param("max_depth",max_depth)

    ## Initialization and train the model
    model = RandomForestClassifier(n_estimators=n_estimators,max_depth=max_depth,random_state=42)
    model.fit(X_train,y_train)

    ## Step 5: Model prediction and log metrics
    y_pred = model.predict(X_test)

    ## Log accuracy
    accuracy = accuracy_score(y_test,y_pred)
    mlflow.log_metric("accuracy",accuracy)

    ## Log classification report
    classification_rep = classification_report(y_test,y_pred,output_dict=True)
    for label,metrics in classification_rep.items():
      if isinstance(metrics,dict):
        for metric,value in metrics.items():
          mlflow.log_metric(f"{label}_{metric}",value)

    ## Log confusion matrix
    conf_matrix = confusion_matrix(y_test,y_pred)
    plt.figure(figsize=(8,6))
    sns.heatmap(conf_matrix,annot=True,fmt='d',cmap="Blues")
    plt.title("confusion matrix")
    plt.xlabel("predicted")
    plt.ylabel("actual")
    plt.savefig("confusion_matrix.png")
    mlflow.log_artifact("confusion_matrix.png")
    plt.close()

    ## Log the model
    mlflow.sklearn.log_model(model,f"random_forest_mode_{vectorizer_name}_{ngram_range}")

## Step 6: Run experiment for Bow and TFidf with different n_grams
ngram_ranges = [(1,1),(1,2),(1,3)]
max_features = 5000

for ngram_range in ngram_ranges:
  ## BOw
  run_experiment("Bow",ngram_range,max_features,vectorizer_name="BOW")
  ## ifidf
  run_experiment("TFIDF",ngram_range,max_features,vectorizer_name='TF-IDF')

2026/02/10 04:36:05 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/usr/local/lib/python3.12/dist-packages/mlflow/models/model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)


🏃 View run BOW_(1, 1)_RandomForest at: https://dagshub.com/codex03080/reddit-sentiement-analysis.mlflow/#/experiments/2/runs/0e9c0d63979c4d7d92838df20828430e
🧪 View experiment at: https://dagshub.com/codex03080/reddit-sentiement-analysis.mlflow/#/experiments/2


2026/02/10 04:37:13 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/usr/local/lib/python3.12/dist-packages/mlflow/models/model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)


🏃 View run TF-IDF_(1, 1)_RandomForest at: https://dagshub.com/codex03080/reddit-sentiement-analysis.mlflow/#/experiments/2/runs/9ea03d65bfe144beb2e694a20bb049c5
🧪 View experiment at: https://dagshub.com/codex03080/reddit-sentiement-analysis.mlflow/#/experiments/2


2026/02/10 04:38:21 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/usr/local/lib/python3.12/dist-packages/mlflow/models/model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)


...

🏃 View run TF-IDF_(1, 2)_RandomForest at: https://dagshub.com/codex03080/reddit-sentiement-analysis.mlflow/#/experiments/2/runs/2ef31d04c9674550abd39e80171f75ed
🧪 View experiment at: https://dagshub.com/codex03080/reddit-sentiement-analysis.mlflow/#/experiments/2


2026/02/10 04:40:36 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/usr/local/lib/python3.12/dist-packages/mlflow/models/model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)


🏃 View run BOW_(1, 3)_RandomForest at: https://dagshub.com/codex03080/reddit-sentiement-analysis.mlflow/#/experiments/2/runs/02bec3f7156e4419bac7f06e620456c1
🧪 View experiment at: https://dagshub.com/codex03080/reddit-sentiement-analysis.mlflow/#/experiments/2


2026/02/10 04:41:45 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/usr/local/lib/python3.12/dist-packages/mlflow/models/model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)


🏃 View run TF-IDF_(1, 3)_RandomForest at: https://dagshub.com/codex03080/reddit-sentiement-analysis.mlflow/#/experiments/2/runs/4b149b85fbca4b2f98a87bc7aa5e2b88
🧪 View experiment at: https://dagshub.com/codex03080/reddit-sentiement-analysis.mlflow/#/experiments/2
